In [1]:
import pandas as pd
import torch
import yaml

from model.crecs import CRECS

/home/tdutton/miniforge3/envs/crecs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
items = pd.read_csv("data/ml-20m/movies.csv")

items = items[["item_id", "item_title"]]

num_items = items["item_id"].nunique()

requests = pd.read_csv('data/ml-20m/requests.csv')
requests = requests.groupby("item_id").agg({
    "item_title": "first",
    "request": list,
}).reset_index()
requests.set_index("item_id", inplace=True, drop=False)

In [3]:
args = yaml.safe_load(open("configs/collaborative.yaml", "r"))

device = "cuda" if torch.cuda.is_available() else "cpu"

args["model"]["recommender"]["num_items"] = requests["item_id"].nunique()

model = CRECS(weights="weights/collaborative/crecs.pt", **args["model"])

In [ ]:
features = (torch.tensor([num_items, 0]), torch.tensor([5.0, 3.5]))

request = "I want to watch a popular romantic comedy"

item_ids, item_ratings = model.predict(features, request)

In [10]:
item_ids

tensor([1033,  357,  148, 3454, 1189,  447, 1044, 1702,  662, 4656])

In [11]:
items.iloc[item_ids]

,item_id,item_title
1033,1033,Raiders of the Lost Ark (Indiana Jones and the...
357,357,True Lies (1994)
148,148,Die Hard: With a Vengeance (1995)
3454,3454,Charlie's Angels (2000)
1189,1189,Die Hard 2 (1990)
447,447,Jurassic Park (1993)
1044,1044,Star Wars: Episode VI - Return of the Jedi (1983)
1702,1702,"Mask of Zorro, The (1998)"
662,662,Twister (1996)
4656,4656,Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...
